In [1]:
import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical computations
import matplotlib.pyplot as plt # For plotting and visualizing data
import re # For working with regular expressions

import nltk # Natural Language Toolkit for text preprocessing
from nltk.corpus import stopwords # List of common stopwords (e.g., "the", "is", etc.)
from nltk.stem.porter import PorterStemmer # For stemming words to their root form

from sklearn.feature_extraction.text import TfidfVectorizer # To convert text to numerical features
from sklearn.model_selection import train_test_split # To split dataset into training and testing
from sklearn.ensemble import RandomForestClassifier # A machine learning model (ensemble method)
from sklearn.metrics import accuracy_score # To evaluate model accuracy
from sklearn.metrics import classification_report # To get precision, recall, F1-score etc.
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # For confusion matrix and visualization

In [2]:
senti=pd.read_csv('train.csv') # To read the training dataset (CSV file) into a pandas DataFrame called 'senti'

In [3]:
senti.head() # To display the first 5 rows of the dataset to inspect its structure

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
senti.shape # To output the shape (rows, columns) of the DataFrame

(27481, 4)

In [5]:
nltk.download('stopwords') # To download the list of English stopwords
print(stopwords.words('english')) # To print the list to verify

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\debar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
senti.isnull().sum() # To count missing (NaN) values for each column

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [7]:
senti=senti.dropna() # To remove rows with any missing values

In [8]:
senti.isnull().sum() # To recheck to ensure no missing values remain

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [9]:
x=senti.drop(columns=['text','textID'],axis=1) # To drop columns not used in modeling
print(x) # To print resulting DataFrame

                                           selected_text sentiment
0                    I`d have responded, if I were going   neutral
1                                               Sooo SAD  negative
2                                            bullying me  negative
3                                         leave me alone  negative
4                                          Sons of ****,  negative
...                                                  ...       ...
27476                                             d lost  negative
27477                                      , don`t force  negative
27478                          Yay good for both of you.  positive
27479                         But it was worth it  ****.  positive
27480  All this flirting going on - The ATG smiles. Y...   neutral

[27480 rows x 2 columns]


In [10]:
porter=PorterStemmer() # To create a PorterStemmer object for stemming words

In [11]:
stop_words=set(stopwords.words('english')) # To convert stopwords to a set for faster lookup

def stemming(selected_text): # Function to clean and stem text
    new_text=re.sub('[^a-zA-Z]',' ',selected_text) # To remove non-letter characters
    new_text=new_text.lower() # To convert to lowercase
    new_text=new_text.split() # To split into words
    new_text=[porter.stem(word) for word in new_text if word not in stop_words] # To stem and remove stopwords
    new_text=' '.join(new_text) # To rejoin words into a single string
    return new_text

In [12]:
senti['selected_text']=senti['selected_text'].apply(stemming) # To apply preprocessing to 'selected_text'

In [13]:
print(senti['selected_text']) # To print the stemmed version of selected text

0                        respond go
1                          sooo sad
2                             bulli
3                         leav alon
4                               son
                    ...            
27476                          lost
27477                          forc
27478                      yay good
27479                         worth
27480    flirt go atg smile yay hug
Name: selected_text, Length: 27480, dtype: object


In [14]:
X=senti['selected_text'].values # Feature: processed text
Y=senti['sentiment'].values # Label: sentiment

In [15]:
print(X) # To print feature values                                     

['respond go' 'sooo sad' 'bulli' ... 'yay good' 'worth'
 'flirt go atg smile yay hug']


In [16]:
print(Y) # To print target sentiment labels

['neutral' 'negative' 'negative' ... 'positive' 'positive' 'neutral']


In [17]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3) # 70% train, 30% test

In [18]:
print(X.shape,X_train.shape,X_test.shape) # To display shapes of full, train, and test datasets

(27480,) (19236,) (8244,)


In [19]:
print(X_train) # To print training feature data

['sad' 'jam session room see star trek'
 'chillen csla wait start watchin pacquiao rerun' ... 'cool movi'
 'wow beauti pictur' 'sorri']


In [20]:
print(X_test) # To print test feature data

['well inspir day room design today schedul head bed' 'forgot fan haha'
 'love' ... 'snugglin tila' 'fair' 'toy stori pwn']


In [21]:
vector=TfidfVectorizer() # To create TF-IDF vectorizer object
vector.fit(X) # To learn vocabulary from all data
X_train=vector.transform(X_train) # To transform training data into vectors
X_test=vector.transform(X_test) # To transform test data

In [22]:
print(X_train) # To print sparse matrix of training data

  (0, 9807)	1.0
  (1, 5901)	0.4989923758561556
  (1, 9692)	0.3896632185942222
  (1, 10018)	0.2795672226059586
  (1, 10084)	0.467177049116599
  (1, 10838)	0.36695978180436206
  (1, 11824)	0.40999692227309764
  (2, 2016)	0.4175637075378642
  (2, 2630)	0.446992727171066
  (2, 8361)	0.446992727171066
  (2, 9513)	0.4175637075378642
  (2, 10851)	0.25211223416420464
  (2, 12488)	0.23525808644616047
  (2, 12552)	0.3643750240524671
  (3, 2251)	1.0
  (4, 1074)	0.46761866814727726
  (4, 2190)	0.4115776353025025
  (4, 2333)	0.3438696333580786
  (4, 2806)	0.3911188682532427
  (4, 4320)	0.39109537406012335
  (4, 9612)	0.27006402727742523
  (4, 10363)	0.3388600994937592
  (5, 3456)	0.4740366023057642
  (5, 7887)	0.5797523660525763
  (5, 8641)	0.5504959243531207
  :	:
  (19228, 8471)	0.35367949348731975
  (19228, 10341)	0.3173396122058412
  (19228, 12233)	0.29174134938499474
  (19228, 12616)	0.251859752871833
  (19229, 745)	0.35749826273882623
  (19229, 1254)	0.4299188390292576
  (19229, 3289)	0.73194

In [23]:
print(X_test) # To print sparse matrix of test data

  (0, 1022)	0.29797611840881305
  (0, 2806)	0.20514415568529845
  (0, 2970)	0.4102636656845808
  (0, 5121)	0.31356516312264326
  (0, 5744)	0.4183696126748751
  (0, 9692)	0.35147596046331475
  (0, 9942)	0.4213933833351253
  (0, 11662)	0.24265842216102643
  (0, 12644)	0.25973591277192803
  (1, 3882)	0.6123894706952528
  (1, 4197)	0.6243181613619168
  (1, 4938)	0.4849803806086012
  (2, 6821)	1.0
  (3, 8094)	0.6642153459740069
  (3, 9205)	0.7475412859318409
  (4, 4567)	0.4444277948424851
  (4, 5121)	0.4489076297981768
  (4, 5385)	0.35030552051569336
  (4, 8120)	0.44299486930567766
  (4, 10631)	0.5310437488055986
  (5, 4330)	1.0
  (6, 7364)	1.0
  (7, 4694)	1.0
  (8, 4126)	0.4492213413546151
  (8, 4715)	0.2748919178126278
  :	:
  (8235, 1231)	0.33665007682526954
  (8235, 1603)	0.32597487839152556
  (8235, 1956)	0.3085967520334949
  (8235, 2276)	0.26480362109241895
  (8235, 2667)	0.39528930573583115
  (8235, 5455)	0.2523460665315504
  (8235, 6615)	0.21234062073774698
  (8235, 8894)	0.32230091

In [ ]:
model=RandomForestClassifier(n_estimators=100)# To initialize with 100 decision trees
model.fit(X_train, Y_train)# To train model on training data

In [ ]:
X_train_pred=model.predict(X_train) # To use the trained model to predict labels for the training data
train_accuracy=accuracy_score(X_train_pred,Y_train) # To calculate training accuracy by comparing predicted and actual labels
print('Accuracy of training data: ',train_accuracy) # To print the computed accuracy score to the console
plt.figure(figsize=(4,4)) # To create a new figure for the plot with width and height=4 inches
plt.bar(['Training Accuracy'],[train_accuracy],color='orange') # To draw a bar chart with a single bar for training accuracy
plt.ylim(0,1) # To set the y-axis limits from 0 to 1 since accuracy is a ratio
plt.ylabel("Accuracy Score") # To label the y-axis
plt.title("Training Accuracy") # To add a title to the bar chart
plt.text(0,train_accuracy-0.05,f"{train_accuracy:.2f}",fontsize=12,color='white',ha='center') # To add a text label above the bar to show the exact accuracy value with 2 decimal places
plt.show() # To display the final plot

In [ ]:
X_test_pred=model.predict(X_test) # To use the trained model to predict sentiment labels for the test data
test_accuracy=accuracy_score(X_test_pred,Y_test) # To calculate test accuracy by comparing predictions to true labels
print('Accuracy of testing data: ',test_accuracy) # To rint the test accuracy to the console
plt.figure(figsize=(4, 4)) # To create a new figure for plotting with 4x4 inch dimensions
plt.bar(['Test Accuracy'],[test_accuracy],color='blue')# To create a bar chart showing test accuracy in blue
plt.ylim(0,1) # To set the vertical axis (y-axis) limits between 0 and 1
plt.ylabel("Accuracy Score") # To label the y-axis as "Accuracy Score"
plt.title("Test Accuracy") # To add a title to the chart
plt.text(0,test_accuracy-0.05,f"{test_accuracy:.2f}",fontsize=12,color='white',ha='center') # To add a text label above the bar to show the exact accuracy value with 2 decimal places
plt.show() # To render and display the final plot

In [ ]:
Y_pred = model.predict(X_test) # To use the trained model to predict sentiment labels on the test dataset
print("Classification Report:\n") # To print a heading to indicate the following output is a classification report
print(classification_report(Y_test,Y_pred)) # To print precision, recall, F1-score, and support for each class

In [ ]:
plt.figure(figsize=(6,4)) # To create a new plot with width 6 inches and height 4 inches
plt.bar(['Train Accuracy','Test Accuracy'],[train_accuracy, test_accuracy],color=['orange', 'blue']) # To plots two bars:One for training accuracy(train_accuracy) in orange and One for test accuracy(test_accuracy) in blue
plt.ylim(0,1) # To set the vertical axis (y-axis) to range from 0 to 1(0% to 100% accuracy)
plt.ylabel("Accuracy Score") # To add a label to the y-axis
plt.title("Train vs Test Accuracy Comparison") # To add a descriptive title to the chart
plt.text(0, train_accuracy - 0.05, f"{train_accuracy:.2f}", ha='center', color='white') # To add a label on(or slightly below)the training accuracy bar with the exact value 
plt.text(1, test_accuracy - 0.05, f"{test_accuracy:.2f}", ha='center', color='white') # To add a label on(or slightly below)the training accuracy bar with the exact value
plt.show() # To render and display the final chart

In [ ]:
labels,counts=np.unique(Y_pred,return_counts=True) # To count how many times each label appears and requency of each class in predictions
plt.figure(figsize=(5,5)) # To create a new square figure for the pie chart (5x5 inches)
plt.pie(counts,labels=labels,autopct='%1.1f%%',startangle=140,colors=['skyblue','lightcoral','red']) # To draw a pie chart depicting distribution of predicted labels
plt.title("Distribution of Predicted Labels") # To add a title to explain the pie chart
plt.axis('equal') # To ensure the pie chart is drawn as a circle (equal aspect ratio)
plt.show() # To render and display the pie chart

In [ ]:
cm=confusion_matrix(Y_test, Y_pred) # To compute the confusion matrix
disp=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_) # To create a ConfusionMatrixDisplay object for plotting.
disp.plot(cmap=plt.cm.Blues) # To render the confusion matrix as a heatmap using a blue color gradient 
plt.title("Confusion Matrix") # To add a title above the plot
plt.show() # To display the final confusion matrix plot

In [ ]:
#Predict sentiment for custom user input
def predict_sentiment(user_input):
    # Preprocess the input like the training data
    processed_input=stemming(user_input)
    vectorized_input=vector.transform([processed_input])
    prediction=model.predict(vectorized_input)[0]
    # Return the predicted sentiment label
    return f"{prediction.capitalize()}😊" if prediction=='positive' else(
           f"{prediction.capitalize()}😐" if prediction=='neutral' else
           f"{prediction.capitalize()}😞")

In [ ]:
#To ask for user input
print("\nSentiment Prediction(Type'exit'to quit)\n")
while True:
    try:
        user_input=input("Enter a sentence: ") # To take input from user
        if user_input.lower()=='exit': # To exit loop if 'exit'
            print("Goodbye!👋")
            break
        result=predict_sentiment(user_input) # To predict sentiment
        print("Predicted Sentiment:",result)
        print("-"*40)
    except Exception as e:
        print("Something went wrong:",e)

DEBARATI_GHOSH